In [6]:
import numpy as np
import cv2 as cv
import os
# Define a class
class CamShift:
    def __init__(self, input_video_path):
        self.input_video = input_video_path
     # Method to start tracking the object 
    def detect(self):
        # video capture object initialization
        cam = cv.VideoCapture(self.input_video)
        # Capture the frame from the webcam 
        re, fr = cam.read()
        # Extract the coordinates of the selected rectangle
        x, y, w, h = 300, 200, 100, 50  
         # Extract the tracking window 
        track_frame = (x, y, w, h)
        # set up the ROI for tracking
        roisetup = fr[y:y + h, x:x + w]
        # Convert the frame to HSV colorspace 
        hsvcolor_roi = cv.cvtColor(roisetup, cv.COLOR_BGR2HSV)
        # Create the mask based on predefined thresholds 
        colormask = cv.inRange(hsvcolor_roi, np.array((0., 60., 32.)), np.array((180., 255., 255.)))
         # Compute the histogram of the region of  
                # interest in the HSV image using the mask 
        roicolor_hist = cv.calcHist([hsvcolor_roi], [0], colormask, [180], [0, 180])
        # Normalize and reshape the histogram 
        cv.normalize(roicolor_hist, roicolor_hist, 0, 255, cv.NORM_MINMAX)
        # Define termination criteria for the tracker 
        termination_crit = (cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT, 10, 1)
        # Iterate until the user presses the Esc key 
        while (1):
            re, fr = cam.read()
            if re == True:
                hsvcolor = cv.cvtColor(fr, cv.COLOR_BGR2HSV)
                 # Compute the histogram back projection 
                dstcolor = cv.calcBackProject([hsvcolor], [0], roicolor_hist, [0, 180], 1)
                # apply CamShift 
                re, track_frame = cv.CamShift(dstcolor, track_frame, termination_crit)
                # Draw it on image
                pts = cv.boxPoints(re)
                pts = np.int0(pts)
                img2 = cv.polylines(fr, [pts], True, 255, 2)
                cv.imshow('img2', img2)
                k = cv.waitKey(30) & 0xff
                if k == 27:
                    break
            else:
                break

In [7]:

### 3. CamShift ###
input_video = os.path.join('videoplayback.mp4')
camshift_obj=CamShift(input_video)
camshift_obj.detect()